In [11]:
!python -m pip install Faker

You should consider upgrading via the 'C:\Users\Egor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [12]:
from numpy.random import choice
import random
from faker import Faker
from faker.providers import phone_number
import json

# Генерируем данные

In [13]:
fake = Faker()
fake.add_provider(phone_number)
pnumbers = [fake.phone_number() for _ in range(100)]
calls = [ {"caller": random.choice(pnumbers), "recipient": random.choice(pnumbers), "duration_s": random.randint(5, 600)} for _ in range(10**3) ]
suspects = choice(pnumbers, size=10, replace=False)
with open("numbers.txt", "w+") as file:
    file.writelines([f'{str(number)}\n' for number in pnumbers])
with open("suspects.txt", "w+") as file:
    file.writelines([f'{str(number)}\n' for number in suspects])
with open("calls.txt", "w+") as file:
    file.writelines([f'caller:{call.get("caller")}|recipient:{call.get("recipient")}|duration_s:{call.get("duration_s")}\n' for call in calls])

# Приводим номера к стандартному виду (10 цифр без знаков)

In [14]:
CHARACTERS_TO_REMOVE = '().-+'

def raw_phone_to_standartized(phone_number):
    """
    Converts raw phone string to standartized one. 
    Phone string considered standartized if and only if it contains only 10 numbers.
    If could not convert right - crashes.
    """
    for char in CHARACTERS_TO_REMOVE:
            phone_number = phone_number.replace(char, '')
    phone_number = phone_number.split('x')[0]
    phone_number = phone_number[-10:]
    int(phone_number)
    assert len(phone_number) == 10
    return phone_number

with open('calls.txt', 'r') as f:
    calls = f.readlines()
with open('suspects.txt', 'r') as f:
    suspects = f.readlines()
with open('new_calls.txt', 'w') as f:
    for call in calls:
        new_call = call
        caller, recipient, duration_s = new_call.split('|')
        caller = 'caller:' + raw_phone_to_standartized(caller.split(':')[1])
        recipient = 'recipient:' + raw_phone_to_standartized(recipient.split(':')[1])
        new_call = caller + '|' + recipient + '|' + duration_s
        f.write(new_call)
with open('new_suspects.txt', 'w') as f:
    for suspect in suspects:
        new_suspect = suspect
        new_suspect = new_suspect.split('\n')[0]
        new_suspect = raw_phone_to_standartized(new_suspect)
        f.write(new_suspect)
        f.write('\n')
with open('new_calls.txt', 'r') as f:
    calls = f.readlines()
with open('new_suspects.txt', 'r') as f:
    suspects = f.readlines()


# Итеративно ищем новых подозреваемых (делал 3 и более звонков подозреваемым из предыдущего круга)

In [15]:
N = 5

with open('new_suspects.txt', 'w') as f:
    current_suspects = suspects.copy()  # саспекты из предыдущего круга
    f.writelines(current_suspects)
    f.write('\n')
    all_suspects = set(suspects)  # все саспекты
    proofs = dict()
    for _ in range(N):
        new_suspects = set()  # саспекты из текущего круга
        for call in calls:
            caller, recipient, duration_s = call.split('|')
            caller = caller.split(':')[1]
            recipient = recipient.split(':')[1]
            duration_s = duration_s.split(':')[1]
            if recipient + '\n' in current_suspects and int(duration_s) > 120:
                try:
                    proofs[caller].append({'recipient': recipient, 'duration_s': int(duration_s)})
                except  KeyError:
                    proofs[caller] = [{'recipient': recipient, 'duration_s': int(duration_s)}]
                if len(proofs[caller]) > 2:
                    new_suspects.add(caller + '\n')
        f.writelines(new_suspects - all_suspects)
        f.write('\n')  # круги между собой отделяем пустой строкой
        all_suspects = all_suspects | new_suspects
        current_suspects = new_suspects.copy()
    with open('proofs.json', 'w') as j_f:
        j_f.write(json.dumps(proofs))

